# DS-SF-23 | Codealong 16 | Exploring Rossmann Drug Store Sales Data (cont.)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import statsmodels as sm
from statsmodels.graphics.tsaplots import plot_acf

pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

%matplotlib inline
plt.style.use('ggplot')

To explore time series models, we will continue to use the Rossmann sales data.

This dataset has sales data for every Rossmann store for a 3-year period and indicators for holidays and basic store information.

In the last class, we saw that we could plot the sales data at a particular store to identify how the sales changed over time.

We also computed autocorrelation for the data at varying lag periods.  This helps us identify if previous timepoints are predictive of future data and which time points are most important - the previous day, week, or month.

In [ ]:
df = pd.read_csv(os.path.join('..', 'datasets', 'rossmann.csv'), skipinitialspace = True, low_memory = False)

df.Date = pd.to_datetime(df.Date)
df.set_index('Date', inplace = True)

df.Sales = df.Sales.astype(float)

Again, let's focus on the first store.

In [ ]:
df = df[df.Store == 1]

In [ ]:
df

Plot the sales over time.

In [ ]:
df[df.Open == 1].Sales.plot()

## Activity | Compute the autocorrelation of Sales in Store 1 for lag 1 and 2.

In [ ]:
# TODO

## Activity | Will we be able to use a predictive model, particularly an autoregressive one?

Answer:

An easier way to diagnose this may be to plot many autocorrelations at once.

In [ ]:
pd.tools.plotting.autocorrelation_plot(df.Sales)

This shows a typical pattern of an autocorrelation plot, that it should decrease to 0 as lag increases.  However, it's hard to observe exactly what the values are.

## `statsmodels` and autocorrelation plots

`statsmodels` has a better autocorrelation plot that allows us to look at fixed number of lag values.

In [ ]:
plot = plot_acf(df.Sales, lags = 10)

Here we observe autocorrelation at 10 lag values.  1 and 2 are what we saw before.  This implies a small but limited impact based on the last few values.  An autoregressive model might be useful.  We also see a larger spike at 7 (the seventh day in the week).

If we observed a handful of random distributed spikes, a moving average model would be useful.

In [ ]:
plot = plot_acf(df.Sales, lags = 25)

Expanding the window to 25 days, we can see that the random spikes occur regularly at 7 days.  What does this mean?

## `statsmodels` and `AR`, `MA`, `ARMA`, and `ARIMA` models

In this class, we will use `statsmodels` to code AR, MA, ARMA, and ARIMA models.

To explore `AR`, `MA`, and `ARMA` models, we will use `sm.tsa.arima_model.ARMA`.  (http://statsmodels.sourceforge.net/0.6.0/generated/statsmodels.tsa.arima_model.ARMA.html)

Remember, an `ARMA` model is a combination of autoregressive and moving average models.

We can train an `AR` model by turning off the MA component (`q = 0`).

In [ ]:
model = sm.tsa.arima_model.ARMA(df[df.Open == 1].Sales, (1, 0)).fit()

model.summary()

By passing `(1, 0)` in the second argument, we are fitting an ARMA model with `p = 1`, `q = 0`.  This is the same as an `AR(1)` model.

In this `AR(1)` model, we learn an intercept (or base sales) value.

Additionally, we learn a coefficient that tells us how to include the latest sales value.

In this case, we add an intercept of ~4800 to 0.68 times the previous month's sales.  Note that the coefficient is not equal to the lag 1 autocorrelation.  This implies the data is __not__ stationary.

We can learn an `AR(2)` model, which regresses each sales value on the last two.

In [ ]:
model = sm.tsa.arima_model.ARMA(df.Sales, (2, 0)).fit()

model.summary()

In this case, we learn two coefficients, which tell us the effect of the last two sales values on the current sales.  While this model may perform better, it may be more difficult to interpret.

## Residuals

To start to diagnose the model, we want to look at residuals.

1. What are residuals?
1. In linear regression, what did we expect of residuals?

- Residuals are the errors of the model or how off our predictions are
- Ideally, we want randomly distributed errors that are small
- If the errors are large, our model does not perform well
- If the errors have a pattern, particularly over time, we may have overlooked something in the model or have periods of time that are different than the rest of the dataset

We can use `statsmodels` to plot the residuals.

In [ ]:
model.resid.plot()

Our model considers a short period of time, so it does not take into account the longer seasonal pattern.  We can also plot the autocorrelations of the residuals.  In an ideal world, these would all be near 0 and appear random.

In [ ]:
plot = plot_acf(model.resid, lags = 50)

This plot shows a problem: the errors are increasing and decreasing every week in a clear pattern.  We may need to expand our model.  To expand this `AR` model to an `ARMA` model, we can include the moving average component as well.

In [ ]:
model = sm.tsa.arima_model.ARMA(df.Sales.astype(float), (1, 1)).fit()

model.summary()

Now we learn two coefficients, one for the `AR(1)` component and one for the `MA(1)` component.

## Activity

1. Take a moment to look at the coefficients of our new model
1. Offer an interpretation of this model

Answer:

We can also use statsmodels to fit `ARIMA` models.  Let's start by using `ARIMA(1, 0, 1)` to fit an `ARMA(1, 1)` model.

In [ ]:
model = sm.tsa.arima_model.ARIMA(df.Sales, (1, 0, 1)).fit()

model.summary()

We can see that this model is the same as our previous ARMA model.  We can also fit a true ARIMA model to predict the difference of the series.

In [ ]:
model = sm.tsa.arima_model.ARIMA(df[df.Open == 1].Sales, (1, 1, 1)).fit()

model.summary()

We can remove the MA component since it does not appear to be useful.

In [ ]:
model = sm.tsa.arima_model.ARIMA(df[df.Open == 1].Sales, (1, 1, 0)).fit()

model.summary()

We now have an `AR(1)` model on the differenced series with a coefficient of -0.18.

## Activity

1. Does this model match the lag 1 autocorrelation of the differenced series?
1. Is the data stationary?

In [ ]:
# TODO

With our models, we can also plot our predictions against the true series using the plot_predict function: We can compare the last 50 days of true values against our predictions.

In [ ]:
plot = model.plot_predict(1, 50)

The function takes two arguments, the start and end index of the dataframe to plot.  Here, we are plotting the last 50 values.  To plot earlier values with our predictions continuing where the true values stop, we can do the following.

In [ ]:
fig, ax = plt.subplots()
ax = df['2014'][df.Open == 1].plot(ax = ax)

fig = model.plot_predict(1, 200, ax = ax, plot_insample = False)

This plots true values in 2014 and our predictions 200 days out from 2014.

## Activity

We can revisit our diagnostics to check that our models are working well.

1. Plot the residuals and autocorrelation of the residuals.
1. Are there patterns or outliers?

Answer:

We can adjust the AR component of the model to adjust for a piece of this.  Let's increase the lag to 7.

In [ ]:
model = sm.tsa.arima_model.ARIMA(df[df.Open == 1].Sales, (7, 1, 2)).fit()

model.summary()

In [ ]:
plot = plot_acf(model.resid, lags = 50)

This removes some of the autocorrelation in the residuals but large discrepancies still exist.

However, they exist where we are breaking our model assumptions.

## Activity

1. Alter the time period of predictions and the `p`, `d`, and `q` parameters
1. Do any of these improve diagnostics?
1. What does changing `p` and `q` imply based upon the autocorrelation plot?
1. How about changing `d`?

Answer:

There are variants of ARIMA that will better handle the seasonal aspect of our data.  This is referred to as Seasonal ARIMA.

These models fit two ARIMA models, one on the current frequency (daily in our example) and another on the seasonal frequency (maybe monthly or yearly patterns).

Additionally, issues with seasonality could be handled by preprocessing tricks such as detrending.